In [1]:
import pandas as pd
import numpy as np
import re
from completing_missing_clinician_questionnaire_2021 import map_additional_column_name_to_2021_clinician_column_name, remove_from_stu_columns, map_additional_column_name_to_2021_student_column_name , fill_id, IsClinician

In [2]:
df_2021 = pd.read_csv(r"../original_data/Schneider Depression Clinic Database.csv", na_values= ' ')
df_2022 = pd.read_csv(r"../original_data/ImmiRiskIPT2022_DATA_2022-12-12_1719.csv", na_values= ' ')

additional_data = pd.read_csv(r"../original_data/Student_Clinician_data_2021.csv", na_values= ' ')
# the warning rooted from the nan values


rename = {'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp', 'cssrs_followup_complete':'cssrs_fw_maya_complete'}
additional_data = additional_data.rename(rename, axis=1)

## error record_id = 363
additional_data = additional_data[additional_data['record_id']!=363]

#do the imputation id like of 2022 dataset
additional_data = fill_id(additional_data)


rename = {'trqsfmarisclin_timestamp': 'trqsfmaris_clin_timestamp', 'trqsfmarisclin_complete': 'trqsfmaris_clin_complete'}

df_2022 = df_2022.rename(rename, axis=1)
rename['chameleon_complete_stu'] = 'chameleon_complete'
df_2021 = df_2021.rename(rename, axis=1)


# consts
mini_start_2022 ='mini_kid_sum_clin_timestamp'
mini_end_2022 = 'mini_kid_sum_clin_complete'

mini_start_add = 'mini_kid_sum_timestamp'
mini_end_add = 'mini_kid_sum_complete'



clinician_start = 'opening_clinicians_complete'
clinician_end_2021_2022 = 'cssrs_t_clin_complete'#, 'cssrs_fw_maya_complete'
clinician_end_add = 'cssrs_t_complete'

student_start = 'opening_students_timestamp'
student_end = 'cssrs_t_stu_complete'


maya_start_2022 = 'cssrs_fw_maya_timestamp'
maya_end_2022 = 'cssrs_fw_maya_complete'


# Chameleon - only student


chameleon_start = 'chameleon_timestamp'

chameleon_end = 'chameleon_complete'

C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44,51,53,99,325,366,439,453,459,464,466,501,507,512,514,521,530,532,569,571,580,599,766,771,777,781,786,788,793,795,804,806,826,843,845,854,873,883,918,941,979,991,1002,1040,1045,1047,1049,1051,1053,1054,1080,1096,1119,1122,1126,1127,1131,1132,1133,1134,1136,1137,1138,1141,1142,1144,1161,1183,1186,1237,1239,1259,1261,1262,1263,1269,1270,1271,1272,1384,1387,1389,1405,1408,1411,1415,1432,1434,1435,1436,1442,1443,1444,1445,1499,1568,1569,1572,1573) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: Dtyp

In [3]:
cols_2021 = list(df_2021.columns)
cols_2022 = list(df_2022.columns)
cols_add = list(additional_data.columns)

### clinician columns
clinician_columns_2021 = cols_2021[cols_2021.index(clinician_start):cols_2021.index(clinician_end_2021_2022)+1]
clinician_columns_2022 = cols_2022[cols_2022.index(clinician_start):cols_2022.index(clinician_end_2021_2022)+1] + cols_2022[cols_2022.index(maya_start_2022):cols_2022.index(maya_end_2022)+1]
clinician_columns_add = cols_add[cols_add.index(clinician_start):cols_add.index(clinician_end_add)+1]


### student columns
stu_columns_2021 = cols_2021[cols_2021.index(student_start):cols_2021.index(student_end)+1]
stu_columns_2022 = cols_2022[cols_2022.index(student_start):cols_2022.index(student_end)+1] #+ cols_2022[cols_2022.index(maya_start_2022):cols_2022.index(maya_end_2022)+1]
stu_columns_add = cols_add[cols_add.index(clinician_start):cols_add.index(clinician_end_add)+1]


## edit additional data student columns
mini_columns_add = cols_add[cols_add.index(mini_start_add):cols_add.index(mini_end_add)+1]
stu_columns_add = [i for i in stu_columns_add if i not in mini_columns_add]
stu_columns_add = [i for i in stu_columns_add if not remove_from_stu_columns(i)]
stu_columns_add = stu_columns_add + ['who', 'who_other', 'name']

stu_columns_add = [i for i in stu_columns_add if i not in ['cgi_i']]

# Chameleon columns
chameleon_columns_2021 = cols_2021[cols_2021.index(chameleon_start):cols_2021.index(chameleon_end)+1]
chameleon_columns_2022 = cols_2022[cols_2022.index(chameleon_start):cols_2022.index(chameleon_end)+1] 
chameleon_columns_add = cols_add[cols_add.index(chameleon_start):cols_add.index(chameleon_end)+1]


In [14]:
map_chameleon_2021_add = {}

for i in range(len(chameleon_columns_add)):
    new_name, success = map_additional_column_name_to_2021_student_column_name(chameleon_columns_add[i], chameleon_columns_2021)
    if success:
        map_chameleon_2021_add[chameleon_columns_add[i]] = new_name

In [15]:
map_for_student_data = {}

for i in range(len(stu_columns_add)):
    new_name, success = map_additional_column_name_to_2021_student_column_name(stu_columns_add[i], stu_columns_2021)
    if success:
        map_for_student_data[stu_columns_add[i]] = new_name

In [16]:
map_for_clinician_data = {}

for i in range(len(clinician_columns_add)):
    new_name, success = map_additional_column_name_to_2021_clinician_column_name(clinician_columns_add[i], clinician_columns_2021)
    if success:
        map_for_clinician_data[clinician_columns_add[i]] = new_name

In [17]:
def fill_columns(columns_to_add, map_for_data, df_2021, row):
    for column in columns_to_add:
        if not row.isna()[column]:
            column_rename = map_for_data[column]

            df_2021.loc[(df_2021['id']==row['id']) & (df_2021['redcap_event_name']==row['redcap_event_name']), column_rename] = row[column]
    return df_2021



# # check if not equals 
# if not (df_2021[(df_2021['id']==row['id']) & (df_2021['redcap_event_name']==row['redcap_event_name'])][column_rename] == row[column]).all():
#     print(column)
#change value


In [18]:
#         only_clin_cols = [column for column in clinician_columns_add if column not in stu_columns_add]
#         print(row[only_clin_cols].isna().all())
        

In [20]:

# cols_2022 = list(df_2022.columns)
# cols_add = list(additional_data.columns)

True

In [21]:
visit_date_stu

NameError: name 'visit_date_stu' is not defined

In [22]:
for i, row in additional_data.iterrows():
    if row['id'] == 'TEST':
        continue
    operator = IsClinician(row)
    
    fill_columns(chameleon_columns_add, map_chameleon_2021_add, df_2021, row)
    
    if operator == 'student':

        fill_columns(stu_columns_add, map_for_student_data, df_2021, row)
    
    elif operator == 'clinician':
        fill_columns(clinician_columns_add, map_for_clinician_data, df_2021, row)


In [10]:
df_2021.to_csv(r'../preprocessed_data/2021_data_imputed_with_clin_stu.csv', index=False)
df_2022.to_csv(r'../preprocessed_data/2022_data_imputed_with_clin_stu.csv', index=False)